# Data preview

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv("/Users/ckkai/Desktop/ML/project/HOSP_DATA.csv")
data

In [ ]:
print(len(data['admission_type'].unique()))
print(data['admission_type'].unique())

In [ ]:
data['admission_type'] = data['admission_type'].astype('category')
data['admission_type'] = data['admission_type'].cat.codes
data

# NLTK Tokenization

In [ ]:
# Tokenization
import nltk
data['symptoms'] = data['symptoms'].apply(nltk.word_tokenize)
data['symptoms']

## The result of removal is bad.

In [ ]:
# Stop word removal
# This might change the meaning after work_tokenize.
# Do not.....-> ......
# In mimiciv database, the symptoms are all important For LOS prediction.

# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
# data['symptoms'] = data['symptoms'].apply(lambda x: [word for word in x if word.lower() not in stop_words])
# data['symptoms']

## Stemming or lemmatization

In [ ]:
# Stemming or lemmatization
# calculi->calculus
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
# data['symptoms'] = data['symptoms'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
# data['symptoms']
data['symptoms'] = data['symptoms'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
data['symptoms']

# SPACY tokenization & punctuation removal

In [ ]:
import spacy
import pandas as pd
import numpy as np
data=pd.read_csv("/Users/ckkai/Desktop/ML/project/HOSP_DATA.csv")
nlp=spacy.load("en_core_web_md")
data['admission_type'] = data['admission_type'].astype('category').cat.codes
data

In [ ]:
gpu = spacy.prefer_gpu()
print(gpu)
nlp = spacy.load('en_core_web_sm')
# define a function to remove punctuation using spacy pipeline
def remove_punctuation(text):
    doc = nlp(text)
    return ' '.join([token.text for token in doc if not token.is_punct])

# apply the remove_punctuation function to the "symptoms" column
data['symptoms'] = data['symptoms'].apply(remove_punctuation)

In [ ]:
data.to_csv('data_remove_punctuation.csv', index=False)

## word2vec 
pre-trained model: https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT

In [1]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [2]:
import spacy
import pandas as pd
import numpy as np
# After remove.
# vectorization=[]
# for i in data["symptoms"]:
#     bert_none_se_padding=tokenizer.encode(
#         text=i,
#         add_special_tokens=Ture,
#         pad_to_max_length = True,return_attention_mask = True
#     )
#     vectorization.append(bert_none_se_padding)

data=pd.read_csv('data_remove_punctuation.csv')
input_ids=[]
attention_masks=[]

for msg in data["symptoms"]:
    bert_inp=tokenizer.encode_plus(msg,add_special_tokens = False,max_length =610,padding='max_length',return_attention_mask = True)#,return_tensors='pt'
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)

In [3]:
len(input_ids)

338675

In [4]:
data['symptoms']= input_ids

In [5]:
data

,subject_id,hadm_id,admission_type,age,BMI,symptoms,target
0,10467237,20000019,5,76,25.366667,177,0
1,10467237,27923538,5,76,25.366667,177,1
2,16925328,20000024,5,84,27.069355,1425,0
3,16925328,21837264,5,84,27.069355,1202,0
4,16925328,21895449,4,84,27.069355,188,0
...,...,...,...,...,...,...,...
338670,16832788,29998928,6,78,28.485714,177,1
338671,19128791,29998991,4,34,28.000000,6272,0
338672,13467099,29999012,5,26,23.700000,12104,0
338673,19435486,29999301,8,38,30.729412,9386,0


## convert numerical type

In [6]:
print("Original\n",data.dtypes)
data['admission_type']=data['admission_type'].astype('float')
data['age']=data['age'].astype('float')
data['target']=data['target'].astype('float')
print("\nFinal\n",data.dtypes)
# data['symptoms']=data['symptoms'].astype('float')
type(data['symptoms'][1])

Original
 subject_id          int64
hadm_id             int64
admission_type      int64
age                 int64
BMI               float64
symptoms            int64
target              int64
dtype: object

Final
 subject_id          int64
hadm_id             int64
admission_type    float64
age               float64
BMI               float64
symptoms            int64
target            float64
dtype: object


numpy.int64

In [7]:
data[['admission_type', 'age', 'BMI']].values

array([[ 5.        , 76.        , 25.36666667],
       [ 5.        , 76.        , 25.36666667],
       [ 5.        , 84.        , 27.06935484],
       ...,
       [ 5.        , 26.        , 23.7       ],
       [ 8.        , 38.        , 30.72941176],
       [ 2.        , 66.        , 24.44      ]])

In [8]:
temp={}
for i in range(len(input_ids)):
    if 610!=len(input_ids[i]):
        temp[i]=input_ids[i]
        print(len(input_ids[i]))

In [9]:
import numpy as np
from sklearn import svm

# concatenate symptoms with other features
X = np.concatenate([data[['admission_type', 'age', 'BMI']].values, np.vstack(input_ids)], axis=1)
y = np.array(data["target"])
# X[1]

In [10]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.98, random_state=42)

# normalization on all input data.

In [17]:
# Normalize the input data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
X_train.shape

(237072, 613)

#### SVM

In [20]:
from sklearn import svm
# C: the smaller , the less over fitting.
# kernel: gausian。
# gamma: the bigger , the more complex the boundry
# degree: improve complex，3 stand for 3 dimension to classify

# kernel='linear' model
svc_linear=svm.SVC(kernel='linear', C=1)
# fit the model
svc_linear.fit(X_train, y_train)
# prediction
predicted1=svc_linear.predict(X_test)
# accuracy
accuracy1 = svc_linear.score(X_test, y_test)
print(predicted1)
print(accuracy1)

0.8038125711806497


In [ ]:
# your implementation for finding parameters
for c in [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]: #[0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]
    for sigma in [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]: # gamma=sigma
        gamma=np.power(sigma,-2.)/2
        model=svm.SVC(C=c,kernel='poly',gamma=gamma)
        model.fit(X_train,y_train)
        current_score=model.score(X_test,y_test)
        if current_score > best_score:
                    best_score = current_score
                    best_params = {'C':c, 'gamma':sigma}

C: 0.1, gamma: 0.001

In [ ]:
print(best_params)

0.817

In [19]:
from sklearn import svm

#  kernel='poly' model
svc_poly=svm.SVC(kernel='poly', gamma='auto', C=1)
# fit the model
svc_poly.fit(X_train[:10000], y_train[:10000])
# prediction
predicted2=svc_poly.predict(X_test)
# accuracy
accuracy2 = svc_poly.score(X_test, y_test)
print(predicted2)
print(accuracy2)

[0. 0. 0. ... 0. 1. 0.]
0.8084406956487505


In [20]:
from sklearn import svm
# C: the smaller , the less over fitting.
# kernel: gausian。
# gamma: the bigger , the more complex the boundry
#  kernel='rbf' model
svc_rbf=svm.SVC(kernel='rbf', gamma='0.7', C=1)
# fit the model
svc_rbf.fit(X_train, y_train)
# predition
predicted3=svc_rbf.predict(X_test)
# accuracy
accuracy3 = svc_rbf.score(X_test, y_test)
print(predicted3)
print(accuracy3)

[0. 0. 0. ... 0. 0. 0.]
0.7892450181077547


## The best kernel polynomial kernel model.

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predicted2,labels=[0,1]))

              precision    recall  f1-score   support

           0       0.82      0.98      0.89     80256
           1       0.66      0.18      0.28     21347

    accuracy                           0.81    101603
   macro avg       0.74      0.58      0.59    101603
weighted avg       0.78      0.81      0.76    101603



## PCA downscale text wordtovector

In [25]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=10)
input_ids_pca=pca.fit_transform(input_ids) 
print(input_ids_pca)

[[ 1.61391224e+04 -3.46954340e+04  1.04097246e+01 ...  2.17773037e+04
   1.50649412e+04 -8.71505628e+03]
 [ 5.20929911e+04 -9.10120237e+03 -2.83461583e+04 ...  1.67976783e+04
   1.34060990e+04  8.66296959e+03]
 [-2.25731188e+04 -7.32998567e+03  8.49665795e+03 ...  1.08359734e+04
   3.43159939e+02 -5.09477423e+03]
 ...
 [-4.71125904e+04  2.91516155e+04 -2.09344707e+04 ...  2.38868414e+03
   3.73049859e+03  7.90132385e+03]
 [-1.45063839e+04 -1.25546090e+04  3.89087024e+03 ... -1.04460935e+04
   3.54166173e+02 -6.49872088e+03]
 [-2.49383227e+04 -9.21076627e+03  1.96686684e+04 ...  4.48147040e+03
   1.13119758e+03  2.24821294e+03]]


In [26]:
X_pca = np.concatenate([data[['admission_type', 'age', 'BMI']].values, np.vstack(input_ids_pca)], axis=1)
y_pca = np.array(data["target"])

In [27]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y_pca, test_size=0.3, random_state=42)

In [29]:
from sklearn import svm

# kernel='poly' model
svc_poly=svm.SVC(kernel='rbf', degree=3, gamma='auto', C=1)
# fit the model
svc_poly.fit(X_train_pca, y_train_pca)
# predict result
predicted4=svc_poly.predict(X_test_pca)
# accuracy
accuracy4 = svc_poly.score(X_test_pca, y_test_pca)
print(predicted4)
print(accuracy4)

[0. 0. 0. ... 0. 0. 0.]
0.7898290404810882


# Note:

It is obvious that, whether we change choose the data with pca, normalization or padding with the different size, the result is still around 80%.

we try Ensemble learning.(reduce the effect of wordtovector.), if the result is the same, we won't apply the model as base model.


# Ensembel learning.

## Homogeneous base learners (voting)

In [5]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 


# use decision tree
tree = DecisionTreeClassifier(random_state=128)
# use SVC
svm = SVC(probability=True, random_state=128)
# use logistic regression
logistic = LogisticRegression(max_iter=10000,random_state=128,penalty='l2')
#max_iter 最大迭代次数，逻辑回归虽然叫回归但是并不使用最小二乘法OLS。逻辑回归往往使用坐标法，梯度下降法以及牛顿法，都是需要迭代的方法。因此如果max_iter,会警告说以达到最大迭代次数，但是还是没有收敛的情况。
# use KNN
Knn = KNeighborsClassifier(n_neighbors=4)
# use GaussianNB
Gsn = GaussianNB()

In [6]:
# Create the ensemble using the VotingClassifier
ensemble = VotingClassifier(
    estimators=[('tree', tree), ('svm', svm), ('logistic', logistic),('KNN', Gsn),('GaussianNB', Knn)],
    voting='hard'  # Use hard voting for majority rule
)

In [7]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# concatenate symptoms with other features
X_ensemble_all = np.concatenate([data[['admission_type', 'age', 'BMI']].values, np.vstack(input_ids)], axis=1)[:12000]
X_ensemble_nemeric = data[['admission_type', 'age', 'BMI']].values[:12000]
X_ensemble_wtv = input_ids[:12000]
y = np.array(data["target"])[:12000]

#scale the data.
X_ensemble_all=scaler.fit_transform(X_ensemble_all)

# Here use KFold, don't split the dataset handly.

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train_ensemble_all, X_test_ensemble_all, y_train_ensemble_all, y_test_ensemble_all = train_test_split(X_ensemble_all, y, test_size=0.3, random_state=42)
X_train_ensemble_nemeric, X_test_ensemble_nemeric, y_train_ensemble_nemeric, y_test_ensemble_nemeric = train_test_split(X_ensemble_nemeric, y, test_size=0.3, random_state=42)
X_train_ensemble_wtv, X_test_ensemble_wtv, y_train_ensemble_wtv, y_test_ensemble_wtv = train_test_split(X_ensemble_wtv, y, test_size=0.3, random_state=42)

In [8]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

from sklearn.model_selection import cross_val_score
kfold=KFold(n_splits=10)
ensemble.fit(X_train_ensemble_all,y_train_ensemble_all)
ensemble_scores=cross_val_score(ensemble, X_ensemble_all, y, cv=kfold,n_jobs=-1)
# Print the mean accuracy of the ensemble classifier
print("Mean Accuracy of Ensemble (Majority Voting):")
print(f"Ensemble: {ensemble_scores.mean():.3f}")

Mean Accuracy of Ensemble (Majority Voting):
Ensemble: 0.799


# The result still stay around 80%

The limitation we supposed is the dimension of text features is too large which causes bias in the distribution of the whole data or the model is not complex enough to fit all the features.